# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [3]:
with open("network_service_sales_data.txt") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [6]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [7]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100


In [8]:
docs[0]

Document(page_content='[客户问题] 我的网络为什么这么慢？ [客服回答] 很抱歉给您带来不便。我可以帮您检查一下网络状态。请问您现在使用的是哪个服务区域？')

In [9]:
len(docs)

30

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [43]:
query = "剩余流量"

In [44]:
answer_list = db.similarity_search(query)

In [45]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我想知道我的剩余流量。 [客服回答] 没问题，我可以帮您查询。请问您能提供一下您的手机号码吗？

[客户问题] 我想查询本月的话费余额。 [客服回答] 当然可以，您可以通过我们的手机APP查询，或者我也可以帮您查看。请问您需要我现在就帮您查询吗？

[客户问题] 我想了解一下贵公司的流量套餐。 [客服回答] 当然可以，我们有多种流量套餐供您选择，根据您的上网习惯，我可以为您推荐最合适的套餐。

[客户问题] 我要更改我的套餐。 [客服回答] 您想要更改成哪种类型的套餐呢？是更多流量还是更多通话时间？请告诉我您的需求，我会为您推荐最合适的套餐。



In [46]:
db.save_local("network_service_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [47]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [48]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019CCC99F190>, search_kwargs={'k': 3})

In [49]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我想知道我的剩余流量。 [客服回答] 没问题，我可以帮您查询。请问您能提供一下您的手机号码吗？

[客户问题] 我想查询本月的话费余额。 [客服回答] 当然可以，您可以通过我们的手机APP查询，或者我也可以帮您查看。请问您需要我现在就帮您查询吗？

[客户问题] 我想了解一下贵公司的流量套餐。 [客服回答] 当然可以，我们有多种流量套餐供您选择，根据您的上网习惯，我可以为您推荐最合适的套餐。



In [50]:
docs = topK_retriever.get_relevant_documents("升级宽带")

In [51]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我的宽带可以升级吗？ [客服回答] 当然可以，我们提供不同的宽带升级选项。请问您目前的主要使用需求是什么？

[客户问题] 我的宽带速度很慢，你们能做什么？ [客服回答] 我理解网速慢会给您带来不便。请问您现在能告诉我您的用户编号或者注册手机号码吗？我将为您检查线路状态并尝试帮您提速。

[客户问题] 我的宽带速度好像不太稳定。 [客服回答] 我能理解您的感受，网络稳定性对于我们来说非常重要。请问您可以提供一下您的用户信息吗？我这边帮您检查一下线路情况。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [52]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [63]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我想知道我的剩余流量。 [客服回答] 没问题，我可以帮您查询。请问您能提供一下您的手机号码吗？



### 提取向量数据库中的`销售回答`

In [54]:
docs = retriever.get_relevant_documents(query)

In [55]:
docs[0].page_content

'[客户问题] 我想知道我的剩余流量。 [客服回答] 没问题，我可以帮您查询。请问您能提供一下您的手机号码吗？'

In [56]:
docs[0].page_content.split("[客服回答] ")

['[客户问题] 我想知道我的剩余流量。 ', '没问题，我可以帮您查询。请问您能提供一下您的手机号码吗？']

In [57]:
ans = docs[0].page_content.split("[客服回答] ")[-1]

In [58]:
ans

'没问题，我可以帮您查询。请问您能提供一下您的手机号码吗？'

#### 尝试各种问题

In [27]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[客服回答] ")[-1] for doc in docs]

    return ans_list

In [28]:
query = "我想离医院近点"

print(sales(query))

[]


d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [29]:
print(sales(query, 0.75))

[]


d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [30]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们将立即查看这笔费用，确保账单的准确性。请问您能提供具体的费用详情或账单时间吗？', '当然可以，我们有多种流量套餐供您选择，根据您的上网习惯，我可以为您推荐最合适的套餐。', '我非常抱歉听到您有这样的体验，我们非常重视您的反馈。为了更好地解决您的问题，请您详细描述一下具体情况，我会将此情况上报给我们的管理部门，并尽力帮您解决。', '我理解您对账单有疑问，让我们一起查看一下详细的账单信息，看看是否有异常的费用。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [31]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [32]:
qa_chain({"query": "你们小区有200万的房子吗？"})

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？', 'result': '我不知道。'}

In [33]:
qa_chain({"query": "小区吵不吵"})

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '小区吵不吵',
 'result': '我不知道这个小区的具体情况，无法判断它是否吵。不同的小区噪音水平可能会有所不同，建议您自己去观察或咨询居住在该小区的人。'}

In [34]:
print(sales("小区吵不吵"))

[]


d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [70]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("network_service_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [71]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [72]:
qa_chain({"query": "剩余流量"})

{'query': '剩余流量',
 'result': '很抱歉，由于我不能直接访问您的手机套餐信息，无法查询您的剩余流量。建议您联系您的手机运营商客服，或者通过手机运营商的手机APP查询剩余流量。'}

In [67]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [69]:
qa_chain({"query": "我想买别墅，应该怎么做"})

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，应该怎么做',
 'result': '购买别墅的过程包括以下几个步骤：\n\n1. 确定预算：首先，您需要确定购买别墅的预算。这将有助于您缩小搜索范围并找到适合您的房产。\n\n2. 选择地点：根据您的需求和喜好，选择一个适合购买别墅的地区。考虑交通、设施、环境等因素。\n\n3. 物色房源：在您选择的地区内寻找合适的房源。您可以通过房地产中介、网络平台或朋友推荐等途径了解房源信息。\n\n4. 实地考察：对感兴趣的房源进行实地考察，了解房屋的实际情况，如建筑结构、装修、周边环境等。\n\n5. 贷款与财务规划：如需贷款购房，提前了解贷款政策和条件，做好财务规划。\n\n6. 签订购房合同：在选定房源后，与卖方签订购房合同，明确双方的权利和义务。\n\n7. 办理产权过户：在签订购房合同后，办理房产过户手续，将房产所有权转移到您的名下。\n\n8. 装修与入住：完成产权过户后，您可以根据需要进行装修，然后入住您的新别墅。\n\n在购买别墅的过程中，建议您咨询专业的房地产经纪人或律师，以确保购房过程顺利进行。'}

In [40]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [41]:
result = qa_chain({"query": "我想买别墅，你们有么"})

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [42]:
result

{'query': '我想买别墅，你们有么', 'result': '抱歉，我不知道。', 'source_documents': []}